In [1]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

In [2]:
torch.manual_seed(42)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cpu


In [4]:
df = pd.read_csv('fashion-mnist_test.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.shape

(10000, 785)

In [6]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [7]:
type(X)

numpy.ndarray

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train = X_train/255.0
X_test = X_test/255.0

In [10]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]


In [11]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [12]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [21]:
class MyNN(nn.Module):

  def __init__(self, num_features):
    super().__init__()

    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(64, 10)
    )

  def forward(self, x):
    return self.model(x)

In [22]:
learning_rate = 0.1
epochs = 100

In [23]:
model = MyNN(X_train.shape[1])
model = model.to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [24]:
for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    y_pred = model(batch_features)

    loss = loss_function(y_pred, batch_labels)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1}, Loss: {avg_loss}')

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch: 1, Loss: 0.8670414868593216
Epoch: 2, Loss: 0.6169831746816635
Epoch: 3, Loss: 0.5550938111543655
Epoch: 4, Loss: 0.5343374823927879
Epoch: 5, Loss: 0.49493185061216355
Epoch: 6, Loss: 0.4686080344319344
Epoch: 7, Loss: 0.4618614014983177
Epoch: 8, Loss: 0.4370156646370888
Epoch: 9, Loss: 0.4272855715155601
Epoch: 10, Loss: 0.40808496075868606
Epoch: 11, Loss: 0.3972875788807869
Epoch: 12, Loss: 0.3882760566174984
Epoch: 13, Loss: 0.37141918268799784
Epoch: 14, Loss: 0.3608652467727661
Epoch: 15, Loss: 0.3571733893454075
Epoch: 16, Loss: 0.3671739955544472
Epoch: 17, Loss: 0.34516108042001725
Epoch: 18, Loss: 0.3265712212324142
Epoch: 19, Loss: 0.31552637642621995
Epoch: 20, Loss: 0.31977213409543037
Epoch: 21, Loss: 0.3273453768789768
Epoch: 22, Loss: 0.3098644694685936
Epoch: 23, Loss: 0.3015074017345905
Epoch: 24, Loss: 0.2941612833738327
Epoch: 25, Loss: 0.2957842037975788
Epoch: 26, Loss: 0.2988521608859301
Epoch: 27, Loss: 0.27441399662196636
Epoch: 28, Loss: 0.27483485625

In [25]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [26]:
correct = 0
total = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:

    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    y_pred = model(batch_features)
    _, predicted = torch.max(y_pred, 1)

    total += batch_labels.size(0)
    correct += (predicted == batch_labels).sum().item()

  accuracy = correct/total * 100
  print(f'Accuracy: {accuracy}')

Accuracy: 86.0


In [27]:
correct = 0
total = 0

with torch.no_grad():
  for batch_features, batch_labels in train_loader:

    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    y_pred = model(batch_features)
    _, predicted = torch.max(y_pred, 1)

    total += batch_labels.size(0)
    correct += (predicted == batch_labels).sum().item()

  accuracy = correct/total * 100
  print(f'Accuracy: {accuracy}')

Accuracy: 98.625
